# Challenge Set 1:  MTA Turnstile Data

# Field Description
http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description.txt
C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS

```
C/A      = Control Area (A002)
UNIT     = Remote Unit for a station (R051)
SCP      = Subunit Channel Position represents an specific address for a device (02-00-00)
STATION  = Represents the station name the device is located at
LINENAME = Represents all train lines that can be boarded at this station
           Normally lines are represented by one character.  LINENAME 456NQR repersents train server for 4, 5, 6, N, Q, and R trains.
DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND   
DATE     = Represents the date (MM-DD-YY)
TIME     = Represents the time (hh:mm:ss) for a scheduled audit event
DESc     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
           1. Audits may occur more that 4 hours due to planning, or troubleshooting activities. 
           2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered. 
ENTRIES  = The comulative entry register value for a device
EXIST    = The cumulative exit register value for a device
```


In [ ]:
from __future__ import print_function, division

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
import datetime

## Exercise 1.1

- Open up a new Jupyter notebook
- Download a few MTA turnstile data files
- Open up a file, use csv reader to read it, make a python dict where
  there is a key for each (C/A, UNIT, SCP, STATION). These are the
  first four columns. The value for this key should be a list of
  lists. Each list in the list is the rest of the columns in a
  row. For example, one key-value pair should look like


{    ('A002','R051','02-00-00','LEXINGTON AVE'):
[
['NQR456', 'BMT', '01/03/2015', '03:00:00', 'REGULAR', '0004945474', '0001675324'],
['NQR456', 'BMT', '01/03/2015', '07:00:00', 'REGULAR', '0004945478', '0001675333'],
['NQR456', 'BMT', '01/03/2015', '11:00:00', 'REGULAR', '0004945515', '0001675364'],
...
]
}

In [5]:
# Source: http://web.mta.info/developers/turnstile.html
def get_data(week_nums):
    url = "turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [180407]
turnstiles_df = get_data(week_nums)
turnstiles_df

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,00:00:00,REGULAR,6566463,2224050
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,04:00:00,REGULAR,6566470,2224053
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,08:00:00,REGULAR,6566470,2224053
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,12:00:00,REGULAR,6566470,2224055
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,16:00:00,REGULAR,6566470,2224055
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,20:00:00,REGULAR,6566470,2224055
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/01/2018,00:00:00,REGULAR,6566470,2224055
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/01/2018,04:00:00,REGULAR,6566470,2224055
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/01/2018,08:00:00,REGULAR,6566470,2224055
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/01/2018,12:00:00,REGULAR,6566470,2224055


In [6]:
turnstiles_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

In [7]:
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]

In [8]:
turnstiles_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [9]:
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,00:00:00,REGULAR,6566463,2224050
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,04:00:00,REGULAR,6566470,2224053
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,08:00:00,REGULAR,6566470,2224053
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,12:00:00,REGULAR,6566470,2224055
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,16:00:00,REGULAR,6566470,2224055


In [10]:
turnstiles_df.tail()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
196983,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,04/06/2018,05:00:00,REGULAR,2869725,385736
196984,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,04/06/2018,09:00:00,REGULAR,2869735,385736
196985,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,04/06/2018,13:00:00,REGULAR,2869816,385739
196986,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,04/06/2018,17:00:00,REGULAR,2869949,385744
196987,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,04/06/2018,21:00:00,REGULAR,2870062,385749


In [11]:
# Three weeks of Data
turnstiles_df.DATE.value_counts().sort_index()

03/31/2018    28129
04/01/2018    28084
04/02/2018    28049
04/03/2018    28189
04/04/2018    28154
04/05/2018    28262
04/06/2018    28121
Name: DATE, dtype: int64


## Exercise 1.2

- Let's turn this into a time series.

 For each key (basically the control area, unit, device address and
 station of a specific turnstile), have a list again, but let the list
 be comprised of just the point in time and the count of entries.

This basically means keeping only the date, time, and entries fields
in each list. You can convert the date and time into datetime objects
-- That is a python class that represents a point in time. You can
combine the date and time fields into a string and use the
[dateutil](https://labix.org/python-dateutil) module to convert it
into a datetime object. For an example check
[this StackOverflow question](http://stackoverflow.com/questions/23385003/attributeerror-when-using-import-dateutil-and-dateutil-parser-parse-but-no).

Your new dict should look something like

{    ('A002','R051','02-00-00','LEXINGTON AVE'):
[
[datetime.datetime(2013, 3, 2, 3, 0), 3788],
[datetime.datetime(2013, 3, 2, 7, 0), 2585],
[datetime.datetime(2013, 3, 2, 12, 0), 10653],
[datetime.datetime(2013, 3, 2, 17, 0), 11016],
[datetime.datetime(2013, 3, 2, 23, 0), 10666],
[datetime.datetime(2013, 3, 3, 3, 0), 10814],
[datetime.datetime(2013, 3, 3, 7, 0), 10229],
...
],
....
}

In [12]:
turnstiles_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [13]:
from datetime import datetime as dt

In [14]:
mask = ((turnstiles_df["C/A"] == "A002") & 
        (turnstiles_df["UNIT"] == "R051") & 
        (turnstiles_df["SCP"] == "02-00-00") & 
        (turnstiles_df["STATION"] == "59 ST"))
turnstiles_df[mask].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,00:00:00,REGULAR,6566463,2224050
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,04:00:00,REGULAR,6566470,2224053
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,08:00:00,REGULAR,6566470,2224053
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,12:00:00,REGULAR,6566470,2224055
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,16:00:00,REGULAR,6566470,2224055


In [15]:
# Take the date and time fields into a single datetime column
turnstiles_df["DATE_TIME"] = pd.to_datetime(turnstiles_df.DATE + " " + turnstiles_df.TIME, format="%m/%d/%Y %H:%M:%S")

In [16]:
mask = ((turnstiles_df["C/A"] == "R626") & 
(turnstiles_df["UNIT"] == "R062") & 
(turnstiles_df["SCP"] == "00-00-00") & 
(turnstiles_df["STATION"] == "CROWN HTS-UTICA"))
turnstiles_df[mask].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
190425,R626,R062,00-00-00,CROWN HTS-UTICA,34,IRT,03/31/2018,00:00:00,REGULAR,13689971,3027500,2018-03-31 00:00:00
190426,R626,R062,00-00-00,CROWN HTS-UTICA,34,IRT,03/31/2018,04:00:00,REGULAR,13689994,3027526,2018-03-31 04:00:00
190427,R626,R062,00-00-00,CROWN HTS-UTICA,34,IRT,03/31/2018,08:00:00,REGULAR,13690166,3027544,2018-03-31 08:00:00
190428,R626,R062,00-00-00,CROWN HTS-UTICA,34,IRT,03/31/2018,12:00:00,REGULAR,13690502,3027606,2018-03-31 12:00:00
190429,R626,R062,00-00-00,CROWN HTS-UTICA,34,IRT,03/31/2018,16:00:00,REGULAR,13690940,3027680,2018-03-31 16:00:00


In [17]:
# turnstiles_df = .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"]).ENTRIES.count().reset_index().sort_values("ENTRIES", ascending=False)

In [18]:
# Sanity Check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
67020,N183,R415,00-00-00,BROAD CHANNEL,2018-04-04 05:00:00,2
0,A002,R051,02-00-00,59 ST,2018-03-31 00:00:00,1
131317,R143,R032,02-00-03,TIMES SQ-42 ST,2018-04-03 16:00:00,1
131319,R143,R032,02-00-03,TIMES SQ-42 ST,2018-04-04 00:00:00,1
131320,R143,R032,02-00-03,TIMES SQ-42 ST,2018-04-04 04:00:00,1


In [19]:
# On 9/16, we seem to have two entries for same time.  Let's take a look
mask = ((turnstiles_df["C/A"] == "R504") & 
(turnstiles_df["UNIT"] == "R276") & 
(turnstiles_df["SCP"] == "00-00-01") & 
(turnstiles_df["STATION"] == "VERNON-JACKSON") &
(turnstiles_df["DATE_TIME"].dt.date == datetime.datetime(2016, 9, 16).date()))
turnstiles_df[mask].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME


* Looks to be a incorrect AUD entry.  May be we should just select the Regular One.

In [20]:
turnstiles_df.DESC.value_counts()

REGULAR       196339
RECOVR AUD       649
Name: DESC, dtype: int64

* Since we are only interested in Entries, we might be OK.

In [21]:
# Get rid of the duplicate entry
turnstiles_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True, ascending=False)
turnstiles_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

In [22]:
# Sanity Check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2018-03-31 00:00:00,1
131197,R143,R032,02-00-00,TIMES SQ-42 ST,2018-04-04 16:00:00,1
131319,R143,R032,02-00-03,TIMES SQ-42 ST,2018-04-04 00:00:00,1
131320,R143,R032,02-00-03,TIMES SQ-42 ST,2018-04-04 04:00:00,1
131321,R143,R032,02-00-03,TIMES SQ-42 ST,2018-04-04 08:00:00,1


* No more duplicate Entries

In [23]:
# Drop Exits and Desc Column.  To prevent errors in multiple run of cell, errors on drop is ignored
turnstiles_df = turnstiles_df.drop(["EXITS", "DESC"], axis=1, errors="ignore")

## Exercise 1.3

- These counts are for every n hours. (What is n?) We want total daily
  entries.

Now make it that we again have the same keys, but now we have a single
value for a single day, which is the total number of passengers that
entered through this turnstile on this day.

In [24]:
turnstiles_daily = turnstiles_df.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"]).ENTRIES.first().reset_index()

In [25]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES
0,A002,R051,02-00-00,59 ST,03/31/2018,6566470
1,A002,R051,02-00-00,59 ST,04/01/2018,6566470
2,A002,R051,02-00-00,59 ST,04/02/2018,6567469
3,A002,R051,02-00-00,59 ST,04/03/2018,6568775
4,A002,R051,02-00-00,59 ST,04/04/2018,6570115


In [26]:
turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"]
                                                       .transform(lambda grp: grp.shift(1)))

In [27]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
0,A002,R051,02-00-00,59 ST,03/31/2018,6566470,NaN,NaN
1,A002,R051,02-00-00,59 ST,04/01/2018,6566470,03/31/2018,6566470.0
2,A002,R051,02-00-00,59 ST,04/02/2018,6567469,04/01/2018,6566470.0
3,A002,R051,02-00-00,59 ST,04/03/2018,6568775,04/02/2018,6567469.0
4,A002,R051,02-00-00,59 ST,04/04/2018,6570115,04/03/2018,6568775.0


In [28]:
turnstiles_daily.tail()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
32920,TRAM2,R469,00-03-01,RIT-ROOSEVELT,04/02/2018,2868821,04/01/2018,2868539.0
32921,TRAM2,R469,00-03-01,RIT-ROOSEVELT,04/03/2018,2869099,04/02/2018,2868821.0
32922,TRAM2,R469,00-03-01,RIT-ROOSEVELT,04/04/2018,2869441,04/03/2018,2869099.0
32923,TRAM2,R469,00-03-01,RIT-ROOSEVELT,04/05/2018,2869706,04/04/2018,2869441.0
32924,TRAM2,R469,00-03-01,RIT-ROOSEVELT,04/06/2018,2870062,04/05/2018,2869706.0


In [29]:
# Drop the rows for last date
turnstiles_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [30]:
turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]].head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
239,A011,R080,01-00-00,57 ST-7 AV,04/01/2018,886024555,03/31/2018,886026296.0
240,A011,R080,01-00-00,57 ST-7 AV,04/02/2018,886021591,04/01/2018,886024555.0
241,A011,R080,01-00-00,57 ST-7 AV,04/03/2018,886017908,04/02/2018,886021591.0
242,A011,R080,01-00-00,57 ST-7 AV,04/04/2018,886014207,04/03/2018,886017908.0
243,A011,R080,01-00-00,57 ST-7 AV,04/05/2018,886010370,04/04/2018,886014207.0


In [31]:
# What's the deal with counter being in reverse
mask = ((turnstiles_df["C/A"] == "A011") & 
(turnstiles_df["UNIT"] == "R080") & 
(turnstiles_df["SCP"] == "01-00-00") & 
(turnstiles_df["STATION"] == "57 ST-7 AV") &
(turnstiles_df["DATE_TIME"].dt.date == datetime.datetime(2016, 8, 27).date()))
turnstiles_df[mask].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,DATE_TIME


* Counter working in Reverse??? - WHHHAT

In [32]:
# Let's see how many stations have this problem

(turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]]
    .groupby(["C/A", "UNIT", "SCP", "STATION"])
     .size())

C/A    UNIT  SCP       STATION        
A011   R080  01-00-00  57 ST-7 AV         6
A025   R023  01-03-01  34 ST-HERALD SQ    6
A049   R088  02-05-00  CORTLANDT ST       6
A066   R118  00-00-00  CANAL ST           6
C021   R212  00-00-02  59 ST              6
D011   R394  01-06-00  BAY PKWY           6
             01-06-01  BAY PKWY           6
H003   R163  01-00-02  6 AV               6
H023   R236  00-06-00  DEKALB AV          6
J034   R007  00-00-02  104 ST             6
K026   R100  00-00-01  METROPOLITAN AV    6
N063A  R011  00-00-04  42 ST-PORT AUTH    6
             00-00-05  42 ST-PORT AUTH    6
             00-00-08  42 ST-PORT AUTH    6
N078   R175  01-03-00  14 ST              6
N137   R354  00-00-00  104 ST             1
             00-06-01  104 ST             6
N203   R195  00-00-01  161/YANKEE STAD    6
N223   R156  01-05-01  BEDFORD PK BLVD    1
N305   R017  01-03-04  LEXINGTON AV/53    6
N327   R254  00-06-01  GRAND-NEWTOWN      6
N342   R019  01-03-02  JAMAICA 179 ST

In [33]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        counter = -counter
    if counter > max_counter:
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
_ = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=1000000)

236 3136681.0
398 2687234.0
838901512 165134.0
55 5119489.0


In [34]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # May be counter is reversed?
        counter = -counter
    if counter > max_counter:
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we are not giving a counter that's too big
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
turnstiles_daily["DAILY_ENTRIES"] = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=1000000)

236 3136681.0
398 2687234.0
838901512 165134.0
55 5119489.0


In [35]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES,DAILY_ENTRIES
1,A002,R051,02-00-00,59 ST,04/01/2018,6566470,03/31/2018,6566470.0,0.0
2,A002,R051,02-00-00,59 ST,04/02/2018,6567469,04/01/2018,6566470.0,999.0
3,A002,R051,02-00-00,59 ST,04/03/2018,6568775,04/02/2018,6567469.0,1306.0
4,A002,R051,02-00-00,59 ST,04/04/2018,6570115,04/03/2018,6568775.0,1340.0
5,A002,R051,02-00-00,59 ST,04/05/2018,6571500,04/04/2018,6570115.0,1385.0


In [36]:
# Verify that we got same results via the non-pandas version.

## Exercise 1.4
We will plot the daily time series for a turnstile.

In ipython notebook, add this to the beginning of your next cell:

    %matplotlib inline

This will make your matplotlib graphs integrate nicely with the
notebook. To plot the time series, import matplotlib with

    import matplotlib.pyplot as plt

Take the list of [(date1, count1), (date2, count2), ...], for the
turnstile and turn it into two lists:
dates and counts. This should plot it:

    plt.figure(figsize=(10,3))
    plt.plot(dates,counts)


## Exercise 1.5

- So far we've been operating on a single turnstile level, let's
  combine turnstiles in the same ControlArea/Unit/Station combo. There
  are some ControlArea/Unit/Station groups that have a single
  turnstile, but most have multiple turnstilea-- same value for the
  C/A, UNIT and STATION columns, different values for the SCP column.

We want to combine the numbers together -- for each
ControlArea/UNIT/STATION combo, for each day, add the counts from each
turnstile belonging to that combo.


## Exercise 1.6

Similarly, combine everything in each station, and come up with a time
series of `[(date1, count1),(date2,count2),...]` type of time series
for each STATION, by adding up all the turnstiles in a station.


## Exercise 1.7

Plot the time series for a station.


## Exercise 1.8

- Make one list of counts for **one** week for one station. Monday's
count, Tuesday's count, etc. so it's a list of 7 counts.
Make the same list for another week, and another week, and another
week.
`plt.plot(week_count_list)` for every `week_count_list` you created
this way. You should get a rainbow plot of weekly commute numbers on
top of each other.


## Exercise 1.9

- Over multiple weeks, sum total ridership for each station and sort
  them, so you can find out the stations with the highest traffic
  during the time you investigate


## Exercise 1.10

- Make a single list of these total ridership values and plot it with

    plt.hist(total_ridership_counts)

to get an idea about the distribution of total ridership among
different stations.

This should show you that most stations have a small traffic, and the
histogram bins for large traffic volumes have small bars.

*Additional Hint*:

If you want to see which stations take the meat of the traffic, you
can sort the total ridership counts and make a `plt.bar` graph. For
this, you want to have two lists: the indices of each bar, and the
values. The indices can just be `0,1,2,3,...`, so you can do

    indices = range(len(total_ridership_values))
    plt.bar(indices, total_ridership_values)